# MCMC CALIBRATION TECHNICS IN CONTEXT OF  INFECTIOUS DISEASE MODELING

## Prerequies

In [ ]:
# pip install summerepi2 estival jax==0.4.24 jaxlib==0.4.24 nevergrad jupyter pymc numpyro

In [ ]:
# pip install multiprocess
!pip install --upgrade --force-reinstall multiprocess


In [2]:
# This is required for pymc parallel evaluation in notebooks

import multiprocess as mp
import platform

if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')

In [3]:
import pandas as pd
from scipy import stats
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"
import summer2
from summer2 import CompartmentalModel # type: ignore
from summer2.parameters import Parameter


# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est
import estival

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
import estival.model
from estival.model import BayesianCompartmentalModel

# Importing the pymc package

import pymc as pm
from estival.wrappers import pymc as epm

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:
esp.GammaPrior?

Init signature: esp.GammaPrior(name: str, shape: float, scale: float, size: int = 1)
Docstring:      A gamma distributed prior
File:           c:\users\abdou\anaconda3\envs\emulearn\lib\site-packages\estival\priors.py
Type:           ABCMeta
Subclasses:     

## Calibration Data

In [5]:
# An example of data for the calibration 

output_labels = {"index": "time", "value": "number infectious"}
data = pd.Series(
    {"active_cases":
     {
         60.0: 3000.0,
         80.0: 8500.0,
         100.0: 21000.0,
         120.0: 40000.0,
         140.0: 44000.0,
         160.0: 30000.0,
         180.0: 16000.0,
         200.0: 7000.0,
     },
    },
)
datafram = pd.DataFrame(data.tolist())#, columns=['active_cases'])
print(data)
#datafram["active_cases"].plot(kind="scatter", labels=output_labels)

active_cases    {60.0: 3000.0, 80.0: 8500.0, 100.0: 21000.0, 1...
dtype: object


In [6]:
# Specify a Truncated normal target with a free dispersion parameter
targets = [
    est.TruncatedNormalTarget("active_cases", data, (0.0,np.inf),
        #esp.UniformPrior("incidence_dispersion",(0.1, obs_noisy.max()*0.1))) #Calibration de l'ecart type
        0.5) #Utiliser un nombre fixe pour ne pas l'inclure dans la calibration
]     


# Uniform priors over our 2 model parameters
priors = [
    esp.UniformPrior("contact_rate", (0.0,0.5)),
    esp.GammaPrior("recovery_rate", 2.0, 0.1),# (0.01,1.0)),
]


In [ ]:
est.TruncatedNormalTarget?

## Model Definition and Configuration

A mechanistic model (ODE-Based) model discribing Infectious Disease transmission.

In [7]:
from models.models import Build_model

In [12]:
def get_sir_model(
    config: dict,
) -> CompartmentalModel:

    model = CompartmentalModel(
        times=(0.0, config["end_time"]),
        compartments=(
            "susceptible", 
            "infectious", 
            "recovered",
        ),
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery",
        fractional_rate=Parameter("recovery_rate"),
        source="infectious",
        dest="recovered",
    )
    model.request_output_for_compartments(name="active_cases", compartments=["infectious"])

    return model

In [13]:
model_config = {
    "population": 1e6,
    "seed": 100.0,
    "end_time": 365.0,
}
parameters = {
    "contact_rate": 0.3,
    "recovery_rate": 0.1,
}

sir_model = get_sir_model(model_config)




In [ ]:
# #sir_model = get_sir_model(model_config)
# sir_model.request_output_for_compartments(
#         "infectious_population_size", "infection"
#     )

In [14]:


sir_model.run(parameters)


pd.DataFrame(
    {
        "modelled": sir_model.get_outputs_df()["infectious"],
        "observed": data["active_cases"],
    }
).plot(kind="scatter", labels=output_labels)

In [15]:
#Defining  a Bayesian Compartmental Model
bcm = BayesianCompartmentalModel(sir_model, parameters, priors, targets)

In [21]:
initial_parameters = {
    "contact_rate": 0.15,
    "recovery_rate": 0.1,
}
bcm.run(initial_parameters)

TypeError: Indexer must have integer or boolean type, got indexer with type float64 at position 0, indexer value []

In [20]:
with pm.Model() as model:
    
    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)
    #variables_defp?
    #print(model.initial_point())

    # The log-posterior value can also be output, but may incur additional overhead
    # Use jacobian=False to get the unwarped value (ie just the 'native' density of the priors
    # without transformation correction factors)
    # pm.Deterministic("logp", model.logp(jacobian=False))
    
    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis sampler
    # See the PyMC docs for more details
    idata_DEM = pm.sample(step=[pm.DEMetropolisZ(variables, S = 0.5,proposal_dist = pm.NormalProposal)], initvals = initial_parameters, draws=1000,tune = 1000 ,cores=4,chains=4)
    #print(model.initial_point()["contact_rate_interval__"])

TypeError: Indexer must have integer or boolean type, got indexer with type float64 at position 0, indexer value []
Apply node that caused the error: BCMLogLike(contact_rate, recovery_rate)
Toposort index: 2
Inputs types: [TensorType(float64, shape=()), TensorType(float64, shape=())]
Inputs shapes: [(), ()]
Inputs strides: [(), ()]
Inputs values: [array(0.25), array(0.2)]
Outputs clients: [['output']]

Backtrace when the node is created (use PyTensor flag traceback__limit=N to make it longer):
  File "c:\Users\abdou\anaconda3\envs\emulearn\lib\site-packages\IPython\core\interactiveshell.py", line 3051, in run_cell
    result = self._run_cell(
  File "c:\Users\abdou\anaconda3\envs\emulearn\lib\site-packages\IPython\core\interactiveshell.py", line 3106, in _run_cell
    result = runner(coro)
  File "c:\Users\abdou\anaconda3\envs\emulearn\lib\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "c:\Users\abdou\anaconda3\envs\emulearn\li